In [ ]:
%cd twitteranalysis
import sqlite3
import environment

# Explore data

## row counts

In [11]:
def get_row_count(db_path):
    conn = sqlite3.connect(db_path)
    r = conn.execute("select count(word) from word_map")
    n = r.fetchone()
    print("%s rows in %s " % (n[0], db_path))
    conn.close()
    return n[0]
get_row_count(environment.MASTER_DB)

16729 rows in /Users/adam/Desktop/TwitterDataAnalysisLogs/master.db 


16729

In [ ]:
# unique words
conn = sqlite3.connect(environment.USER_DB_MASTER)
r = conn.execute("select count(distinct word) from word_map_deux")
n = r.fetchone()
print("%s unique words in the master user db" % n[0])
conn.close()

In [ ]:
def master_row_generator():
    conn = sqlite3.connect(environment.MASTER_DB)
    r = conn.execute("select * from word_map_deux")
    while True:
        yield r.fetchone()

def db_row_generator(filepath):
    conn = sqlite3.connect(filepath)
    r = conn.execute("select * from word_map_deux")
    while True:
        yield r.fetchone()


In [ ]:
gen = db_row_generator(environment)

In [ ]:
for i in range(0, 22):
    print(next(gen)[2])

with exceptions in filter
    - 156590

without 
    - 164499

with regex
    - 144807, 145514

## Word search

In [ ]:
def get_user_ids_for_word(word, db=environment.USER_DB_NO_STOP):
    query = """
          SELECT m.user_id AS uid, 
          m.word_index - 1 AS wix
          FROM word_map_deux m
          WHERE m.word=?
    """
    conn = sqlite3.connect(db)
    s = (word,)
    r = conn.execute(query, s)
    return r.fetchall()
    conn.close()

In [ ]:
r = get_user_ids_for_word('remo')

In [ ]:
len(r)

In [ ]:
def get_preceeding_words(word, db=environment.USER_DB_NO_STOP):
    query = """
          SELECT a.word AS rwid, a.user_id
          FROM word_map_deux a
          JOIN (
              SELECT m.user_id AS uid, 
              m.word_index - 1 AS wix
              FROM word_map_deux m
              WHERE m.word=?
              ) AS b
            ON (a.user_id = b.uid)
            WHERE a.word_index = b.wix
        """
    conn = sqlite3.connect(db)
    s = (word,)
    r = conn.execute(query, s)
    return r.fetchall()
    conn.close()

In [ ]:
r = get_preceeding_word('remo')

In [ ]:
len(r)

In [ ]:
r

In [47]:

def get_preceeding_word_counts(word, cutoff=None, db=environment.USER_DB_NO_STOP):
    query = """
    SELECT a.word, count(a.word)
          FROM word_map a
          JOIN (
              SELECT m.user_id AS uid, 
              m.word_index - 1 AS wix
              FROM word_map m
              WHERE m.word=?
              ) AS b
            ON (a.user_id = b.uid)
            WHERE a.word_index = b.wix
            group by a.word
        """
    conn = sqlite3.connect(db)
    s = (word,)
    r = conn.execute(query, s)

    if cutoff is None:
        return r.fetchall()
    return [x for x in r if x[1] >= cutoff]

In [54]:
%%time

result = get_preceeding_word_counts('pain', cutoff=10)

CPU times: user 12.9 s, sys: 955 ms, total: 13.9 s
Wall time: 14.5 s


In [55]:
len(result)

91

In [56]:
result

[('acute', 37),
 ('advanced', 15),
 ('alleviate', 14),
 ('alleviating', 10),
 ('ankle', 18),
 ('arthritis', 21),
 ('back', 489),
 ('based', 11),
 ('best', 14),
 ('body', 14),
 ('cancer', 12),
 ('certified', 17),
 ('chronic', 1610),
 ('comprehensive', 14),
 ('constant', 20),
 ('drug-free', 11),
 ('du', 12),
 ('effective', 13),
 ('eliminate', 15),
 ('emotional', 13),
 ('every', 11),
 ('experience', 12),
 ('facial', 21),
 ('family', 11),
 ('feel', 33),
 ('fibromyalgia', 15),
 ('foot', 31),
 ('free', 20),
 ('general', 12),
 ('get', 10),
 ('health', 18),
 ('heel', 22),
 ('help', 17),
 ('hip', 10),
 ('holistic', 10),
 ('https', 18),
 ('interventional', 76),
 ('intractable', 17),
 ('joint', 102),
 ('knee', 39),
 ('know', 22),
 ('le', 12),
 ('less', 31),
 ('let', 14),
 ('life', 14),
 ('limb', 11),
 ('live', 21),
 ('living', 19),
 ('love', 23),
 ('manage', 12),
 ('medical', 10),
 ('mom', 16),
 ('much', 19),
 ('multidisciplinary', 10),
 ('muscle', 31),
 ('musculoskeletal', 22),
 ('natural', 49),

In [44]:
b

[('alcohol', 23),
 ('coffee', 11),
 ('drug', 28),
 ('new', 11),
 ('sex', 11),
 ('slight', 14),
 ('unhealthy', 14)]

Addict (>30)

[('beauty', 48),
 ('blogger', 44),
 ('book', 138),
 ('caffeine', 116),
 ('cat', 42),
 ('chocolate', 83),
 ('coffee', 690),
 ('disney', 42),
 ('enjoy', 32),
 ('fashion', 66),
 ('fitness', 88),
 ('food', 98),
 ('football', 40),
 ('gym', 105),
 ('instagram', 38),
 ('internet', 62),
 ('love', 106),
 ('lover', 97),
 ('makeup', 61),
 ('media', 129),
 ('movie', 98),
 ('music', 340),
 ('netflix', 35),
 ('new', 41),
 ('news', 65),
 ('polish', 31),
 ('porn', 31),
 ('recovering', 35),
 ('series', 42),
 ('sex', 42),
 ('shoe', 52),
 ('sports', 77),
 ('starbucks', 76),
 ('tattoo', 65),
 ('tea', 182),
 ('travel', 80),
 ('tv', 200),
 ('twitter', 79),
 ('visit', 62),
 ('wife', 39),
 ('writer', 58),
 ('youtube', 44)]

Addiction (>10)
[('alcohol', 23),
 ('coffee', 11),
 ('drug', 28),
 ('new', 11),
 ('sex', 11),
 ('slight', 14),
 ('unhealthy', 14)]

Pain >10
[('acute', 37),
 ('advanced', 15),
 ('alleviate', 14),
 ('alleviating', 10),
 ('ankle', 18),
 ('arthritis', 21),
 ('back', 489),
 ('based', 11),
 ('best', 14),
 ('body', 14),
 ('cancer', 12),
 ('certified', 17),
 ('chronic', 1610),
 ('comprehensive', 14),
 ('constant', 20),
 ('drug-free', 11),
 ('du', 12),
 ('effective', 13),
 ('eliminate', 15),
 ('emotional', 13),
 ('every', 11),
 ('experience', 12),
 ('facial', 21),
 ('family', 11),
 ('feel', 33),
 ('fibromyalgia', 15),
 ('foot', 31),
 ('free', 20),
 ('general', 12),
 ('get', 10),
 ('health', 18),
 ('heel', 22),
 ('help', 17),
 ('hip', 10),
 ('holistic', 10),
 ('https', 18),
 ('interventional', 76),
 ('intractable', 17),
 ('joint', 102),
 ('knee', 39),
 ('know', 22),
 ('le', 12),
 ('less', 31),
 ('let', 14),
 ('life', 14),
 ('limb', 11),
 ('live', 21),
 ('living', 19),
 ('love', 23),
 ('manage', 12),
 ('medical', 10),
 ('mom', 16),
 ('much', 19),
 ('multidisciplinary', 10),
 ('muscle', 31),
 ('musculoskeletal', 22),
 ('natural', 49),
 ('neck', 150),
 ('nerve', 21),
 ('neuropathic', 31),
 ('new', 10),
 ('pain', 86),
 ('pediatric', 38),
 ('pelvic', 74),
 ('persistent', 30),
 ('physical', 27),
 ('providing', 17),
 ('real', 11),
 ('reduce', 47),
 ('reducing', 12),
 ('regional', 76),
 ('relieve', 63),
 ('relieves', 11),
 ('relieving', 17),
 ('responsible', 11),
 ('severe', 15),
 ('shoulder', 29),
 ('specialist', 14),
 ('spinal', 15),
 ('spine', 16),
 ('sports', 11),
 ('stop', 13),
 ('therapy', 10),
 ('time', 13),
 ('topical', 11),
 ('treating', 25),
 ('true', 11),
 ('us', 20),
 ('w/chronic', 10),
 ('wants', 19),
 ('without', 39)]

# Create stopword free db

In [1]:
%cd twitteranalysis
import sqlite3
from nltk.corpus import stopwords
import environment

(bookmark:twitteranalysis) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis
/Users/adam/Dropbox/PainNarrativesLab/TwitterDataAnalysis


In [3]:
english_stops = stopwords.words('english')

def make_stopword_exclusion_query(stops):
    st = " AND word != '%s'"
    query = "SELECT * FROM word_map WHERE word != '2' "
    for s in stops:
        n = st % s
        query += n 
    return query


def row_generator(source_db, query):
    conn = sqlite3.connect(source_db)
    try:
        r = conn.execute(query)
        while True:
            v = r.fetchone()
            if v is None: raise StopIteration
            yield v
    finally:
        conn.close()


query = make_stopword_exclusion_query(english_stops)


In [25]:
def remove_old_table_name(db):
    q = "alter table word_map_deux rename to word_map"
    conn = sqlite3.connect(db)
    conn.execute(q)
    conn.commit()
    conn.close()

In [27]:
def remove_stopwords(source_db, target_db):
    query = make_stopword_exclusion_query(english_stops)

    conn2 = sqlite3.connect(target_db)

    try:
        userQuery = """INSERT INTO word_map (tweet_id, user_id, word, sentence_index, word_index) VALUES (?, ?, ?, ?, ?)"""
        conn2.executemany( userQuery, row_generator(source_db, query) )
        conn2.commit()
    except StopIteration:
        print('stopped')
    finally:
        conn2.close()
        print('done')

    
target_db = environment.USER_DB_NO_STOP
source_db = 

/Users/adam/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: generator 'row_generator' raised StopIteration
  if __name__ == '__main__':


j



gen = row_generator(environment.USER_DB_MASTER, query)
conn2 = sqlite3.connect(environment.USER_DB_NO_STOP)

size = 1000
queue = []

cnt = 0


def save_rows(rows, conn):
#     print('save_rows called; count is %s' % cnt)
    userQuery = """INSERT INTO word_map (tweet_id, user_id, word, sentence_index, word_index) VALUES (?, ?, ?, ?, ?)"""
    conn.executemany( userQuery, rows )
    conn.commit()


prev = 'j'
try:

    while True:    
        cnt += 1
        try:
            if len(queue) >= size:
                save_rows(queue, conn2)
                queue = []
            v = next(gen)
            prev = v
            if v is not None:
                queue.append(v)

        except StopIteration:
            save_rows(queue, conn2)
            
            break

    print('jip')
    
except Exception as e:
    print(e)
    print(prev)
    
finally:
    conn2.close()
    print("count is %s" % cnt)


In [21]:
get_row_count(test)

12107 rows in /Users/adam/Desktop/TwitterDataAnalysisLogs/user-test.db 


12107

In [22]:
get_row_count(environment.MASTER_DB) - get_row_count(test)

16729 rows in /Users/adam/Desktop/TwitterDataAnalysisLogs/master.db 
12107 rows in /Users/adam/Desktop/TwitterDataAnalysisLogs/user-test.db 


4622

0 rows in /Users/adam/Desktop/TwitterDataAnalysisLogs/user-test.db 


0

In [ ]:
15510416 - n[0]